In [4]:
import cv2
import mediapipe as mp
import numpy as np
from collections import deque
import time

print("✅ Librerías cargadas")

✅ Librerías cargadas


In [5]:
print("""
🎯 DETECTOR DE SWIPES

Algoritmo:
1. MediaPipe detecta la muñeca (landmark 0)
2. Calculamos la posición actual vs anterior
3. Si el desplazamiento > umbral → SWIPE
4. Determinamos dirección: horizontal (LEFT/RIGHT) o vertical (UP/DOWN)
5. Suavizamos con historial para evitar falsos positivos

Parámetros clave:
- SPEED_THRESHOLD: Velocidad mínima para considerar swipe (0.1 = sensible, 0.3 = rígido)
- SMOOTHING_FRAMES: Frames para confirmar gesto (3-7)
""")


🎯 DETECTOR DE SWIPES

Algoritmo:
1. MediaPipe detecta la muñeca (landmark 0)
2. Calculamos la posición actual vs anterior
3. Si el desplazamiento > umbral → SWIPE
4. Determinamos dirección: horizontal (LEFT/RIGHT) o vertical (UP/DOWN)
5. Suavizamos con historial para evitar falsos positivos

Parámetros clave:
- SPEED_THRESHOLD: Velocidad mínima para considerar swipe (0.1 = sensible, 0.3 = rígido)
- SMOOTHING_FRAMES: Frames para confirmar gesto (3-7)



In [6]:
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

hands = mp_hands.Hands(
    static_image_mode=False,
    max_num_hands=1,
    min_detection_confidence=0.7,
    min_tracking_confidence=0.7
)

print("✅ MediaPipe inicializado")

✅ MediaPipe inicializado


In [9]:
# Test: ver coordenadas de la muñeca en tiempo real
cap = cv2.VideoCapture(0)

print("📹 Moviendo la mano, observa las coordenadas")
print("Presiona 'q' para salir\n")

prev_x = None
prev_y = None

for _ in range(100):  # Solo 100 frames para test
    ret, frame = cap.read()
    if not ret:
        break
    
    frame = cv2.flip(frame, 1)
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(rgb)
    
    if results.multi_hand_landmarks:
        wrist = results.multi_hand_landmarks[0].landmark[0]
        
        if prev_x is not None:
            delta_x = wrist.x - prev_x
            delta_y = wrist.y - prev_y
            speed = np.sqrt(delta_x**2 + delta_y**2)
            
            print(f"Pos: ({wrist.x:.3f}, {wrist.y:.3f}) | Delta: ({delta_x:+.3f}, {delta_y:+.3f}) | Speed: {speed:.3f}")
            
            # Visualizar
            cv2.putText(frame, f"Speed: {speed:.3f}", (10, 40),
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)
        
        prev_x = wrist.x
        prev_y = wrist.y
    else:
        prev_x = None
        prev_y = None
    
    cv2.imshow('Test Coordenadas', frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

print("\n✅ Test completado - observa qué valores de speed salen con movimientos rápidos")

📹 Moviendo la mano, observa las coordenadas
Presiona 'q' para salir

Pos: (0.690, 0.688) | Delta: (-0.039, -0.041) | Speed: 0.056
Pos: (0.660, 0.646) | Delta: (-0.030, -0.042) | Speed: 0.051
Pos: (0.629, 0.587) | Delta: (-0.030, -0.059) | Speed: 0.066
Pos: (0.609, 0.553) | Delta: (-0.020, -0.034) | Speed: 0.040
Pos: (0.598, 0.538) | Delta: (-0.011, -0.015) | Speed: 0.019
Pos: (0.593, 0.532) | Delta: (-0.005, -0.006) | Speed: 0.008
Pos: (0.593, 0.531) | Delta: (-0.001, -0.000) | Speed: 0.001
Pos: (0.595, 0.540) | Delta: (+0.003, +0.008) | Speed: 0.009
Pos: (0.599, 0.551) | Delta: (+0.004, +0.012) | Speed: 0.012
Pos: (0.604, 0.569) | Delta: (+0.005, +0.017) | Speed: 0.018
Pos: (0.609, 0.586) | Delta: (+0.005, +0.017) | Speed: 0.018
Pos: (0.615, 0.599) | Delta: (+0.006, +0.013) | Speed: 0.015
Pos: (0.620, 0.608) | Delta: (+0.005, +0.009) | Speed: 0.010
Pos: (0.625, 0.614) | Delta: (+0.005, +0.006) | Speed: 0.008
Pos: (0.630, 0.618) | Delta: (+0.005, +0.003) | Speed: 0.006
Pos: (0.633, 0.6

In [10]:
class SwipeDetector:
    """
    Detector especializado en 4 movimientos rápidos direccionales
    """
    def __init__(self, speed_threshold=0.15, smoothing_frames=5):
        """
        Args:
            speed_threshold: Velocidad mínima para detectar swipe (0.1-0.3)
            smoothing_frames: Frames para suavizar (3-7)
        """
        self.mp_hands = mp.solutions.hands
        self.hands = self.mp_hands.Hands(
            static_image_mode=False,
            max_num_hands=1,
            min_detection_confidence=0.7,
            min_tracking_confidence=0.7
        )
        
        self.speed_threshold = speed_threshold
        self.gesture_history = deque(maxlen=smoothing_frames)
        
        self.prev_wrist_x = None
        self.prev_wrist_y = None
        self.prev_time = None
        
    def detect_swipe(self, wrist_x, wrist_y):
        """
        Detecta swipe basado en movimiento de muñeca
        
        Returns:
            str: "SWIPE_LEFT", "SWIPE_RIGHT", "SWIPE_UP", "SWIPE_DOWN", None
        """
        current_time = time.time()
        
        # Primera iteración
        if self.prev_wrist_x is None:
            self.prev_wrist_x = wrist_x
            self.prev_wrist_y = wrist_y
            self.prev_time = current_time
            return None
        
        # Calcular desplazamiento
        delta_x = wrist_x - self.prev_wrist_x
        delta_y = wrist_y - self.prev_wrist_y
        delta_time = current_time - self.prev_time
        
        # Evitar división por cero
        if delta_time < 0.001:
            return None
        
        # Calcular velocidad
        speed = np.sqrt(delta_x**2 + delta_y**2) / delta_time
        
        # Actualizar posición previa
        self.prev_wrist_x = wrist_x
        self.prev_wrist_y = wrist_y
        self.prev_time = current_time
        
        # Detectar swipe si velocidad > umbral
        if speed > self.speed_threshold:
            # Determinar dirección
            if abs(delta_x) > abs(delta_y):
                # Movimiento horizontal
                return "SWIPE_RIGHT" if delta_x > 0 else "SWIPE_LEFT"
            else:
                # Movimiento vertical
                return "SWIPE_DOWN" if delta_y > 0 else "SWIPE_UP"
        
        return None
    
    def process_frame(self, frame):
        """
        Procesa frame y retorna gesto detectado
        
        Returns:
            tuple: (gesture, hand_detected, results)
        """
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = self.hands.process(rgb_frame)
        
        gesture = None
        hand_detected = False
        
        if results.multi_hand_landmarks:
            hand_detected = True
            wrist = results.multi_hand_landmarks[0].landmark[0]
            
            # Detectar swipe
            swipe = self.detect_swipe(wrist.x, wrist.y)
            
            # Suavizar con historial
            if swipe:
                self.gesture_history.append(swipe)
                
                # Confirmar si hay consistencia
                if len(self.gesture_history) >= 3:
                    most_common = max(set(self.gesture_history),
                                    key=self.gesture_history.count)
                    if self.gesture_history.count(most_common) >= 2:
                        gesture = most_common
        else:
            # Reset si no hay mano
            self.prev_wrist_x = None
            self.prev_wrist_y = None
            self.gesture_history.clear()
        
        return gesture, hand_detected, results
    
    def close(self):
        """Cierra MediaPipe"""
        self.hands.close()

print("✅ SwipeDetector creado")

✅ SwipeDetector creado


In [11]:
# Experimento: encontrar el mejor speed_threshold

print("🧪 CALIBRACIÓN DE SENSIBILIDAD\n")
print("Prueba diferentes umbrales:")

thresholds = [0.10, 0.15, 0.20, 0.25]

for threshold in thresholds:
    print(f"\n{'='*50}")
    print(f"Probando threshold = {threshold}")
    print(f"{'='*50}")
    print("Haz swipes rápidos y observa qué detecta")
    print("Presiona 'q' para probar siguiente umbral\n")
    
    detector = SwipeDetector(speed_threshold=threshold)
    cap = cv2.VideoCapture(0)
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        frame = cv2.flip(frame, 1)
        gesture, hand_detected, results = detector.process_frame(frame)
        
        # UI
        cv2.rectangle(frame, (0, 0), (640, 80), (0, 0, 0), -1)
        q
        text = f"Threshold: {threshold}"
        cv2.putText(frame, text, (10, 30),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)
        
        if gesture:
            cv2.putText(frame, f"GESTO: {gesture}", (10, 70),
                       cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 255, 0), 3)
            print(f"✅ {gesture}")
        elif hand_detected:
            cv2.putText(frame, "Haz un swipe rápido...", (10, 70),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 0), 2)
        
        cv2.imshow('Calibración', frame)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()
    detector.close()

print("\n✅ Calibración completada - elige el threshold que mejor funcionó")

🧪 CALIBRACIÓN DE SENSIBILIDAD

Prueba diferentes umbrales:

Probando threshold = 0.1
Haz swipes rápidos y observa qué detecta
Presiona 'q' para probar siguiente umbral

✅ SWIPE_UP
✅ SWIPE_UP
✅ SWIPE_UP
✅ SWIPE_UP
✅ SWIPE_UP
✅ SWIPE_UP
✅ SWIPE_LEFT
✅ SWIPE_LEFT
✅ SWIPE_LEFT
✅ SWIPE_LEFT
✅ SWIPE_DOWN
✅ SWIPE_DOWN
✅ SWIPE_DOWN
✅ SWIPE_DOWN
✅ SWIPE_DOWN
✅ SWIPE_DOWN
✅ SWIPE_DOWN
✅ SWIPE_DOWN
✅ SWIPE_DOWN
✅ SWIPE_DOWN
✅ SWIPE_DOWN
✅ SWIPE_DOWN
✅ SWIPE_DOWN
✅ SWIPE_DOWN
✅ SWIPE_DOWN
✅ SWIPE_DOWN
✅ SWIPE_DOWN
✅ SWIPE_DOWN
✅ SWIPE_DOWN
✅ SWIPE_DOWN
✅ SWIPE_DOWN
✅ SWIPE_DOWN
✅ SWIPE_DOWN
✅ SWIPE_DOWN
✅ SWIPE_DOWN
✅ SWIPE_DOWN
✅ SWIPE_DOWN
✅ SWIPE_DOWN
✅ SWIPE_DOWN
✅ SWIPE_RIGHT
✅ SWIPE_RIGHT
✅ SWIPE_RIGHT
✅ SWIPE_RIGHT
✅ SWIPE_RIGHT
✅ SWIPE_RIGHT
✅ SWIPE_LEFT
✅ SWIPE_LEFT
✅ SWIPE_LEFT
✅ SWIPE_LEFT
✅ SWIPE_LEFT
✅ SWIPE_UP
✅ SWIPE_UP
✅ SWIPE_UP
✅ SWIPE_UP
✅ SWIPE_UP
✅ SWIPE_UP
✅ SWIPE_UP
✅ SWIPE_UP
✅ SWIPE_UP
✅ SWIPE_UP
✅ SWIPE_UP
✅ SWIPE_UP
✅ SWIPE_LEFT
✅ SWIPE_LEFT
✅ SWIPE_LEFT
✅ SWIPE_UP
✅ SWI